In [ ]:
# Install Kaggle library
!pip install -q kaggle

import os
os.environ['KAGGLE_USERNAME'] = "katetuzina" # username from the json file
os.environ['KAGGLE_KEY'] = "7578fa8dde4c34ef17a35fab34c9b088" # key from the json file
#!kaggle datasets download -d iarunava/happy-house-dataset # api copied from kaggle
!kaggle datasets download -d datamunge/sign-language-mnist

In [ ]:
!ls

In [ ]:
!unzip sign-language-mnist.zip

In [ ]:
!ls

In [ ]:
import pandas as pd
df = pd.read_csv('sign_mnist_train.csv')
print(df)

testDF = pd.read_csv('sign_mnist_test.csv')
print(testDF)


In [ ]:
def getData(df):
  y = df["label"].values.reshape((df.shape[0], 1)) 
  x = df.drop(columns="label").values.reshape((df.shape[0], 28, 28, 1))
  print(y.shape, x.shape)
  return x, y

x, y = getData(df)
tX, tY = getData(testDF)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=.25)

In [ ]:
from tensorflow.keras import layers, models, Sequential

model = Sequential([
    layers.Conv2D(64, kernel_size=3, activation='relu', input_shape=(28,28,1)),
    layers.Conv2D(32, kernel_size=5, activation='relu'),
    layers.Dropout(0.25),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.Dropout(0.25),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.Dropout(0.25),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(500, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(25, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
import tensorflow as tf

model.fit(x_train, y_train, validation_split=0.1, epochs=10)

In [ ]:
model.evaluate(tX,  tY)

In [ ]:
model = Sequential([
    layers.Conv2D(64, kernel_size=3, activation='relu', input_shape=(28,28,1)),
    layers.Conv2D(32, kernel_size=5, activation='relu'),
    layers.Dropout(0.25),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.Dropout(0.25),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.Dropout(0.25),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(500, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(25, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


datagen = ImageDataGenerator(rotation_range=20, zoom_range=0.15,
	width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15,
	horizontal_flip=True, fill_mode="nearest")

train_generator = datagen.flow(x_train, y_train, batch_size=32)
val_generator = datagen.flow(x_val, y_val, batch_size=32)
test_generator = datagen.flow(tX, tY, batch_size=32)

In [ ]:


model.fit_generator(
  train_generator,
  epochs=10,
  validation_data=(x_val,y_val), steps_per_epoch=len(x_train) // 32)

In [ ]:
model.evaluate(tX,  tY)

In [ ]:
from keras.applications import MobileNet
from keras.layers import GlobalAveragePooling2D, Dense
from keras.models import Model

base_model=MobileNet(weights='imagenet',include_top=False) 

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) 
x=Dense(1024,activation='relu')(x) 
x=Dense(512,activation='relu')(x)
preds=Dense(25,activation='softmax')(x) 
model=Model(inputs=base_model.input,outputs=preds)

In [ ]:
print(len(model.layers))
#for layer in model.layers:
#    layer.trainable=False

# or if we want to set the first 20 layers of the network to be non-trainable
for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

In [ ]:
import numpy as np
def transform(dataset):
  newDataset = list()
  for x in dataset:
    x = np.repeat(x, 3, 2)
    newDataset.append(x)
  return np.array(newDataset)

newTrainX = transform(x_train)
newValX = transform(x_val)

print(newTrainX.shape, newValX.shape)

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(newTrainX, y_train, validation_data=(newValX, y_val), epochs=10)